In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as web
from datetime import timedelta
from datetime import datetime
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from scipy import stats
import seaborn as sns
import yfinance as yf
import math

In [2]:
def getData(portfolio):
    df = pd.DataFrame()
    for stock in portfolio:
        s = yf.Ticker(stock)
        df[stock] = s.history(period='max')["Close"]
    return df

In [3]:
def VaR(portfolio, weights, price, date = datetime.today()):
    '''
    Takes in the list of stocks in your portfolio, 
    '''
    df = getData(portfolio)
    df = df[-501:]
    df_exp =(df)/df.iloc[0]
    df_exp = df_exp*weights*price
    df['Value'] = df_exp.sum(axis = 1)
    df_loss = df.set_index(np.arange(0,501,1))
    for i in range(1,501):
        df_loss.iloc[i-1] = (df.iloc[i]/df.iloc[i-1])*df.iloc[-1]
    df_loss = df_loss[:-1]
    for i in range (500):
        df_loss['Value'].iloc[i]  = round(df_loss["Value"].iloc[i]-df["Value"].iloc[-1] , 2)
    arr = df_loss['Value'].values *-1
    arr = np.sort(arr)
    print("The 1 day 99 percent confidence VaR is: ",'{:2f}'.format(round(arr[4],2)*-1))
    print("The 10 day 99 percent confidence VaR is: ",'{:2f}'.format(round(arr[4],2)*math.sqrt(10)*-1))


In [12]:
portfolio = ['AMZN']
weights = np.array([1])
price = 10000000
VaR(portfolio, weights, price)


The 1 day 99 percent confidence VaR is:  1186213.420000
The 10 day 99 percent confidence VaR is:  3751136.198258


In [10]:

portfolio = ['FB', "AAPL", "AMZN", 'NFLX', 'GOOG']
weights = np.array([0.2,0.2,0.2,0.2,0.2])
start = '2013-01-01'
VaR(portfolio, weights, price)


The 1 day 99 percent confidence VaR is:  888754.780000
The 10 day 99 percent confidence VaR is:  2810489.386162


Value at Risk is the defined as the maximum loss that we are anticipating the N number of days with X% confidence. 

Here we can clearly see that if we invest the same amount of money(10 million dollars) in one stock(AMZN), the value at risk is much higher as compared to the diverse portfolio withing FAANG. This is still not the most optimal portfolio. We can optimize the portfolio and that would lead to even lower value at Risk. 

Value at Risk is a great indicator of the fact that diversification in portfolio is very important. 